In [1]:

import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import io
#import torchvision.transforms.functional as F


In [2]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms 
from torch.utils.data import Dataset

In [3]:
from torchvision.models.detection import keypointrcnn_resnet50_fpn
from torchvision.models.vgg import vgg16_bn

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters
#num_classes = 10
learning_rate = 1e-3
batch_size = 16
num_epochs = 30

cuda


In [5]:
model = vgg16_bn(pretrained=True)

In [6]:
model.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=1000, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=1000, out_features=500, bias=True),
    nn.Linear(in_features=500, out_features=224, bias=True),
    nn.Linear(in_features=224, out_features=14, bias=True),
    )

In [7]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [8]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, lable_dir, img_dir, transform=None):
        self.annotations = lable_dir
        self.root_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(os.listdir(self.annotations))

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, 'train_' + str(index) + '.png')
        lables_path = os.path.join(self.annotations, 'train_' + str(index) + '.txt')
        image = io.imread(img_path)
        target_data = []
        with open(lables_path, 'r') as f:
            lines_list = f.readlines()         # all lines as list

            
            for j in range(3, 10): 
                string = lines_list[j]
                str1, str2 = string.split(' ')
                x_ = float(str1)
                y_ = float(str2)
                target_data.append(x_)
                target_data.append(y_)

                # if j == 9:
                #     target_data = torch.FloatTensor(target_data)
                #     lables_dataset.append(target_data)  
                #     #target_data.clear()
                #     print(len(lables_dataset))
        y_label = torch.FloatTensor(target_data)

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [9]:
cwd = 'C:/Users/info/Desktop/Ear_Landmarks/ear_detection_rcnn/model_parameters/'

dataset = 'C:/Users/info/Desktop/Ear_Landmarks/ear_detection_rcnn/data/dataset/images'
la = 'C:/Users/info/Desktop/Ear_Landmarks/ear_detection_rcnn/data/dataset/lables'




#train_dataset = cwd.transforms.ToTensor()

In [10]:
dataset = CatsAndDogsDataset(
    lable_dir= la,
    img_dir= dataset,
    transform=transforms.ToTensor(),
)

In [11]:
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [12]:
#train_loader = DataLoader(dataset=sample,  batch_size=batch_size, shuffle=True)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
for epoch in range(num_epochs):
    losses = []

    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

    
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

        

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")
    if epoch > 4 :
        torch.save(model.state_dict(), cwd + 'model' +str(epoch) + '.pt')

Cost at epoch 0 is 0.01103
Cost at epoch 1 is 0.00930
Cost at epoch 2 is 0.01089
Cost at epoch 3 is 0.01080
Cost at epoch 4 is 0.01033
Cost at epoch 5 is 0.00898
Cost at epoch 6 is 0.00864
Cost at epoch 7 is 0.00864
Cost at epoch 8 is 0.00819
Cost at epoch 9 is 0.00783
Cost at epoch 10 is 0.00765
Cost at epoch 11 is 0.00804
Cost at epoch 12 is 0.01279
Cost at epoch 13 is 0.07571
Cost at epoch 14 is 0.08954
Cost at epoch 15 is 0.08857
Cost at epoch 16 is 0.08868
Cost at epoch 17 is 0.08851
Cost at epoch 18 is 0.08832
Cost at epoch 19 is 0.08849
Cost at epoch 20 is 0.08861
Cost at epoch 21 is 0.08855
Cost at epoch 22 is 0.08847
Cost at epoch 23 is 0.08834
Cost at epoch 24 is 0.08855
Cost at epoch 25 is 0.08833
Cost at epoch 26 is 0.08827
Cost at epoch 27 is 0.08833
Cost at epoch 28 is 0.08837
Cost at epoch 29 is 0.08826


In [ ]:

#model.load_state_dict(torch.load(cwd + '15.pth') )
#model.eval()